In [ ]:
PYMEPIX_IP = '127.0.0.1'
PYMEPIX_PORT = 5056

In [ ]:
# Imports
import time
import numpy as np

import panel as pn
import param


from functools import partial


import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.figure import Figure

#<debug
pn.extension('terminal')
debug = pn.widgets.Debugger(name='My Debugger')
#debug>

from pymepix.channel.client import Client
from pymepix.channel.channel_types import ChannelDataType, Commands

In [ ]:
pn.extension(nthreads=4)

In [ ]:
_EVENT = 0
_X = 1
_Y = 2
_TOF = 3
_TOT = 4

class OnlineProcessClass(param.Parameterized):
    
    """Constructor"""
    def __init__(self, **params):
        super(OnlineProcessClass, self).__init__(**params)
        
    """ROI"""    
    Element_ROI = {
              "H2O": ((5.5, 5.7), 'red'),
              "H3O": ((6.2, 6.5), 'blue'),
              "W2":  ((7.0, 7.5), 'green'),
              "W2H": ((7.5, 8.5), 'yellow'),}
    
    """varivables for data processing/visualisation"""    
    
    H_VMI = np.zeros((255, 255))
    ROI_H_VMI = {i:np.zeros((255, 255),) for i in Element_ROI.keys()}
    H_VMI_bg = np.zeros((255, 255))
    bins_VMI = (range(0, 256), range(0, 256))
    
    bins_tof = np.linspace(0, 10, 500)
    x_axe = bins_tof[:-1]
    tof = np.zeros(len(bins_tof)-1)
    tof_accu = np.zeros(len(bins_tof)-1)
   
    clear_data = False
    
    
    """parametrized fields"""    

    status = param.String(default="Started", doc="Status")
    
    roi_select = param.ListSelector(default=[], 
                                      objects=Element_ROI.keys(), label='ROI')
    
    in_data = param.Dict(precedence=-1) #parameter containing the data received from client
    
    reset_button = param.Action(lambda x: x.param.trigger('reset_button'), label='reset_data')
    
    """figures"""
    fig_histo2d = Figure()#figsize=(8, 6))
    ax_histo2d = fig_histo2d.subplots()
    im_histo2d = ax_histo2d.imshow(H_VMI)
    ax_histo2d.invert_yaxis()
    
    fig_tof = Figure()#figsize=(8, 6))
    ax_tof = fig_tof.subplots()
    plot_tof = ax_tof.plot(x_axe, tof)



   
    
    def process_data(self):
       
        if self.clear_data:
            self.tof_accu = np.zeros(len(self.x_axe))
            self.H_VMI = np.zeros((255, 255))
            for roi_name in self.ROI_H_VMI.keys():
                self.ROI_H_VMI[roi_name] = np.zeros((255,255),)
            self.clear_data = False


        np_arr = self.in_data['data']
        mcs_tof = np_arr[_TOF]*1e6

        """update the roi histograms"""
        for area_name in self.Element_ROI.keys():
            tof_range = self.Element_ROI[area_name][0]
            filt_indxs = np.logical_and(mcs_tof>tof_range[0], mcs_tof<tof_range[1])
            new_2d_histo_ranged = np.histogram2d(np_arr[_X][filt_indxs], 
                                                 np_arr[_Y][filt_indxs], bins=self.bins_VMI)[0]
            self.ROI_H_VMI[area_name] += new_2d_histo_ranged


        new_2d_histo = np.histogram2d(np_arr[_X], np_arr[_Y], bins=self.bins_VMI)[0]
        new_histo = np.histogram(mcs_tof, bins=self.bins_tof)[0]

        self.tof_accu += new_histo
        max_tof_value = np.max(self.tof_accu)

        self.tof = self.tof_accu / max_tof_value 
        self.H_VMI += new_2d_histo
        
        
        #self.ax_histo2d.imshow(np.log(self.H_VMI.swapaxes(0,1) + 1.0))        
        #self.ax_histo2d.invert_yaxis()        
        #display(pn.panel(self.fig_histo2d))
        #print('here')
        
        """function to process the received commands from pymepix"""
    def process_command(self):
        pass
        
    def view_2d_histo(self):
        self.ax_histo2d.clear()
        
        if self.roi_select != []:
            histo_sum = np.zeros((255, 255))
            for area_name in self.roi_select:
                histo_sum += self.ROI_H_VMI[area_name]
            self.ax_histo2d.imshow(np.log(histo_sum.swapaxes(0,1) + 1.0))
        else:        
            self.ax_histo2d.imshow(np.log(self.H_VMI.swapaxes(0,1) + 1.0))
        
        self.ax_histo2d.invert_yaxis()
        
        return self.fig_histo2d
    
        
    def view_tof_histo(self):
        self.ax_tof.clear()
        self.ax_tof.plot(self.x_axe, self.tof)        
                
        self.ax_tof.set_ylabel('Normalized amplitude')
        self.ax_tof.set_xlabel('ToF, [mcsec]')
        
        """draw roi lines"""
        for area in self.roi_select:
            tof_range = self.Element_ROI[area][0]
            color = self.Element_ROI[area][-1]
            #self.fig_tof.patches.extend([plt.Rectangle((tof_range[0],0),
            #                    tof_range[1],1.0, fill=True, color='g', alpha=0.5, zorder=1000,
            #                    transform=self.fig_tof.transFigure, figure=self.fig_tof)])
            self.ax_tof.plot((tof_range[0], tof_range[0]),(0,1), color=color)
            self.ax_tof.plot((tof_range[1], tof_range[1]),(0,1), color=color)
            
        
        return self.fig_tof
   
                            
    
    def panel(self):
        return pn.Column(self.view_2d_histo, self.view_tof_histo)
    
    

  
    """function called on in_data change"""  
    @param.depends('in_data', watch=True)
    def _update_boolean(self):
        if self.in_data['type'] == ChannelDataType.COMMAND.value:
            try:
                self.process_command()
            except:
                raise Exception('Processing exception')
        else:
            self.process_data()
        
    @param.depends('reset_button')
    def reset_data(self):
        self.clear_data = True
        
def callback_func(_instance, _data_filter, _data):    
    if _data['type'] in _data_filter:
        _instance.in_data = _data
        
instance = OnlineProcessClass()

pn.Row(pn.WidgetBox(instance), instance.reset_data, instance.panel()).servable()


In [ ]:
client = Client((PYMEPIX_IP, PYMEPIX_PORT), partial(callback_func, instance, 
                            [ChannelDataType.TOF.value, 
                             ChannelDataType.COMMAND.value],))

In [ ]:
#pn.Row(instance.reset_data, instance.panel()).servable()